In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun  1 17:31:40 2020

@author: manav

Modifed on 23 AUG 2022

by mahayasa adiputra
"""

import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from imblearn.over_sampling import SMOTE
import sklearn.metrics as mt
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NeighbourhoodCleaningRule
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from numpy import mean
from numpy import std
from imblearn.combine import SMOTEENN
from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score
import math
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# record start time
start = time.time()

le = LabelEncoder()
# Importing the dataset
dataset=pd.read_csv('https://raw.githubusercontent.com/mahayasa/various-sampling-churn-prediction/main/data/mobile-churn.csv')
dataset=dataset.dropna()
#dataset=dataset.fillna(0)

#transform data into numeric value

#filling missing value with mean
#dataset['TotalCharges'].replace(to_replace = 0, value = dataset['TotalCharges'].mean(), inplace=True)
X=dataset.drop(['churn','year','month','user_account_id'],axis=1)
y=dataset["churn"]

#imblearn SMOTE
#smote_enn = SMOTE()
#smote_enn = SMOTEENN()
#X, y = smote_enn.fit_resample(X, y)

#smote_enn = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
#X, y = smote_enn.fit_resample(X, y)
#ncr = NeighbourhoodCleaningRule(n_neighbors=5, kind_sel='all')
#X, y = ncr.fit_resample(X, y)


#kfold cross validation
cv = KFold(n_splits=5, random_state=1, shuffle=True)

end = time.time()

print("The time of execution of preprocess:",
      (end-start), "s")


The time of execution of preprocess: 3.335519552230835 s


In [ ]:
# decision tree
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('none-tn-mobile-dt-cs-auc.xlsx', index=False)



Best: 0.778227 using {'class_weight': {0: 100, 1: 10}}
0.778227 (0.004778) with: {'class_weight': {0: 100, 1: 10}}
0.777005 (0.004348) with: {'class_weight': {0: 100, 1: 20}}
0.776888 (0.003360) with: {'class_weight': {0: 100, 1: 30}}
0.775548 (0.004291) with: {'class_weight': {0: 100, 1: 40}}
0.775879 (0.005023) with: {'class_weight': {0: 100, 1: 50}}
0.777331 (0.003318) with: {'class_weight': {0: 100, 1: 60}}
0.776919 (0.004602) with: {'class_weight': {0: 100, 1: 70}}
0.776909 (0.003398) with: {'class_weight': {0: 100, 1: 80}}
0.777145 (0.002868) with: {'class_weight': {0: 100, 1: 90}}
0.777277 (0.004298) with: {'class_weight': {0: 100, 1: 100}}


ValueError: mount failed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define the source file path (in Colab)
source_file = '/content/none-tn-mobile-dt-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/"

# Copy the file to the destination
!cp $source_file $destination_folder

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('none-tn-mobile-dt-cs-f1.xlsx', index=False)

Best: 0.756835 using {'class_weight': {0: 100, 1: 90}}
0.608785 (0.003776) with: {'class_weight': {0: 100, 1: 10}}
0.610777 (0.004928) with: {'class_weight': {0: 100, 1: 20}}
0.717315 (0.004650) with: {'class_weight': {0: 100, 1: 30}}
0.720202 (0.005023) with: {'class_weight': {0: 100, 1: 40}}
0.724363 (0.005477) with: {'class_weight': {0: 100, 1: 50}}
0.728591 (0.004325) with: {'class_weight': {0: 100, 1: 60}}
0.753119 (0.003389) with: {'class_weight': {0: 100, 1: 70}}
0.755387 (0.002808) with: {'class_weight': {0: 100, 1: 80}}
0.756835 (0.003036) with: {'class_weight': {0: 100, 1: 90}}
0.756144 (0.004019) with: {'class_weight': {0: 100, 1: 100}}


In [ ]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('none-tn-mobile-dt-cs-gmean.xlsx', index=False)

Best: 0.763580 using {'class_weight': {0: 100, 1: 90}}
0.509497 (0.005478) with: {'class_weight': {0: 100, 1: 10}}
0.514659 (0.006768) with: {'class_weight': {0: 100, 1: 20}}
0.689639 (0.006640) with: {'class_weight': {0: 100, 1: 30}}
0.695006 (0.006410) with: {'class_weight': {0: 100, 1: 40}}
0.699480 (0.007290) with: {'class_weight': {0: 100, 1: 50}}
0.706980 (0.005968) with: {'class_weight': {0: 100, 1: 60}}
0.759262 (0.004229) with: {'class_weight': {0: 100, 1: 70}}
0.760922 (0.004162) with: {'class_weight': {0: 100, 1: 80}}
0.763580 (0.003147) with: {'class_weight': {0: 100, 1: 90}}
0.763240 (0.003919) with: {'class_weight': {0: 100, 1: 100}}


In [ ]:
# LR
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('none-tn-mobile-lr-cs-auc.xlsx', index=False)

Best: 0.878798 using {'class_weight': {0: 100, 1: 90}}
0.871501 (0.003492) with: {'class_weight': {0: 100, 1: 10}}
0.872904 (0.003358) with: {'class_weight': {0: 100, 1: 20}}
0.875365 (0.003438) with: {'class_weight': {0: 100, 1: 30}}
0.875492 (0.003496) with: {'class_weight': {0: 100, 1: 40}}
0.876650 (0.003287) with: {'class_weight': {0: 100, 1: 50}}
0.877684 (0.003187) with: {'class_weight': {0: 100, 1: 60}}
0.878212 (0.002972) with: {'class_weight': {0: 100, 1: 70}}
0.878373 (0.003179) with: {'class_weight': {0: 100, 1: 80}}
0.878798 (0.003249) with: {'class_weight': {0: 100, 1: 90}}
0.878771 (0.002964) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('none-tn-mobile-lr-cs-f1.xlsx', index=False)

Best: 0.760714 using {'class_weight': {0: 100, 1: 100}}
0.442131 (0.000638) with: {'class_weight': {0: 100, 1: 10}}
0.443084 (0.000764) with: {'class_weight': {0: 100, 1: 20}}
0.714869 (0.006516) with: {'class_weight': {0: 100, 1: 30}}
0.714985 (0.006365) with: {'class_weight': {0: 100, 1: 40}}
0.716813 (0.010914) with: {'class_weight': {0: 100, 1: 50}}
0.735389 (0.017904) with: {'class_weight': {0: 100, 1: 60}}
0.747291 (0.006872) with: {'class_weight': {0: 100, 1: 70}}
0.749815 (0.007200) with: {'class_weight': {0: 100, 1: 80}}
0.754613 (0.004961) with: {'class_weight': {0: 100, 1: 90}}
0.760714 (0.005785) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('none-tn-mobile-lr-cs-gmean.xlsx', index=False)

Best: 0.699875 using {'class_weight': {0: 100, 1: 100}}
0.018599 (0.015140) with: {'class_weight': {0: 100, 1: 10}}
0.038957 (0.009332) with: {'class_weight': {0: 100, 1: 20}}
0.607922 (0.008663) with: {'class_weight': {0: 100, 1: 30}}
0.609095 (0.008451) with: {'class_weight': {0: 100, 1: 40}}
0.613152 (0.019275) with: {'class_weight': {0: 100, 1: 50}}
0.651585 (0.033821) with: {'class_weight': {0: 100, 1: 60}}
0.675334 (0.009861) with: {'class_weight': {0: 100, 1: 70}}
0.679930 (0.010180) with: {'class_weight': {0: 100, 1: 80}}
0.688008 (0.007051) with: {'class_weight': {0: 100, 1: 90}}
0.699875 (0.009307) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#SVM
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
model = SVC()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance1)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('none-mobile-svm-cs-auc.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/none-mobile-svm-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/mobile"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.883881 using {'class_weight': {0: 40, 1: 100}}
0.865217 (0.003207) with: {'class_weight': {0: 10, 1: 100}}
0.877612 (0.005432) with: {'class_weight': {0: 20, 1: 100}}
0.881663 (0.004004) with: {'class_weight': {0: 30, 1: 100}}
0.883881 (0.004042) with: {'class_weight': {0: 40, 1: 100}}
0.881580 (0.003946) with: {'class_weight': {0: 50, 1: 100}}
0.876464 (0.004144) with: {'class_weight': {0: 60, 1: 100}}
0.864394 (0.004163) with: {'class_weight': {0: 70, 1: 100}}
0.858148 (0.005118) with: {'class_weight': {0: 80, 1: 100}}
0.856199 (0.005380) with: {'class_weight': {0: 90, 1: 100}}
0.854572 (0.004462) with: {'class_weight': {0: 100, 1: 100}}


In [ ]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('none-mobile-svm-cs-f1.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/none-mobile-svm-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/mobile"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.798242 using {'class_weight': {0: 60, 1: 100}}
0.711041 (0.003275) with: {'class_weight': {0: 10, 1: 100}}
0.778221 (0.004192) with: {'class_weight': {0: 20, 1: 100}}
0.785825 (0.005708) with: {'class_weight': {0: 30, 1: 100}}
0.792660 (0.004024) with: {'class_weight': {0: 40, 1: 100}}
0.796103 (0.003973) with: {'class_weight': {0: 50, 1: 100}}
0.798242 (0.003996) with: {'class_weight': {0: 60, 1: 100}}
0.792016 (0.004049) with: {'class_weight': {0: 70, 1: 100}}
0.792169 (0.003942) with: {'class_weight': {0: 80, 1: 100}}
0.790086 (0.004259) with: {'class_weight': {0: 90, 1: 100}}
0.790290 (0.004363) with: {'class_weight': {0: 100, 1: 100}}


In [ ]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('none-mobile-svm-cs-gmean.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/none-mobile-svm-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/mobile"

# Copy the file to the destination
!cp $source_file $destination_folder